## Problem Description
___
Determine the optimal length and quantity of space needed in 1000's of sq.ft. to meet space requirements will minimizing total cost of the leases. Leases can be for any amount for time periods from 1 to 6 months.

Required space  

| Jan | Feb | Mar | Apr | May | Jun | Jul | Aug |
|-----|-----|-----|-----|-----|-----|-----|-----|
| 24  | 28  | 31  | 28  | 27  | 25  | 34  | 28  |

Cost of Lease (in months)

|  1  |  2  |  3  |  4  |  5  |  6  |
|-----|-----|-----|-----|-----|-----|
| 400 | 780 | 1140| 1480| 1800| 2100|





In [1]:
import numpy as np 
import pandas as pd
import pulp

In [7]:
#Fixed variables

space_required = [24, 28, 31, 28, 27, 25, 34, 28]
costs = [400, 780, 1140, 1480, 1800, 2100]
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug']
lease = [1, 2, 3, 4, 5, 6]

n_months = len(months)
n_lease = len(lease)


In [46]:
#Create the model
variable_name = [month+str(l) for month in months for l in lease]

model = pulp.LpProblem(name='lease_optimization', sense=pulp.LpMinimize)

lease_variables = pulp.LpVariable.matrix('L', variable_name,
                                               lowBound=0, cat=pulp.LpInteger)

allocation = np.array(lease_variables).reshape(n_months,n_lease)

#the total of january leases must be greater than or equal to the space required for january
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(allocation[0,:]),
    sense=pulp.LpConstraintGE,
    name='Jan_inv_req',
    rhs=space_required[0]))

#the total of February leases and leases made in January that include february must be greater than or equal to 
#the space required for February
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(allocation[1,:])+pulp.lpSum(allocation[0,1:]),
    sense=pulp.LpConstraintGE,
    name='Feb_inv_req',
    rhs=space_required[1]))

#the total of March leases and leases made in Jan/Feb that include march must be greater than or equal to 
#the space required for march
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(allocation[2,:])+pulp.lpSum(allocation[0,2:])+pulp.lpSum(allocation[1,1:]),
    sense=pulp.LpConstraintGE,
    name='Mar_inv_req',
    rhs=space_required[2]))

#the total of Apr through Aug leases and leases made in Jan/Feb/Mar that include apr must be greater than or equal to 
#the space required for apr
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(allocation[3,:-1])+pulp.lpSum(allocation[0,3:])+pulp.lpSum(allocation[1,2:])+pulp.lpSum(allocation[2,1:]),
    sense=pulp.LpConstraintGE,
    name='Apr_inv_req',
    rhs=space_required[3]))

#the total of May through Aug leases and leases made in Jan/Feb/Mar/Apr that include apr must be greater than or equal to 
#the space required for May
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(allocation[4,:-2])+pulp.lpSum(allocation[0,4:])+pulp.lpSum(allocation[1,3:])+pulp.lpSum(allocation[2,2:])+pulp.lpSum(allocation[3,1:-1]),
    sense=pulp.LpConstraintGE,
    name='May_inv_req',
    rhs=space_required[4]))

#the total of Jun through Aug leases and leases made in Jan/Feb/Mar/Apr/May that include apr must be greater than or equal to 
#the space required for Jun
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(allocation[5,:-3])+pulp.lpSum(allocation[0,5])+pulp.lpSum(allocation[1,4:])+pulp.lpSum(allocation[2,3:])+
        pulp.lpSum(allocation[3,2:-1])+pulp.lpSum(allocation[4,1:-2]),
    sense=pulp.LpConstraintGE,
    name='Jun_inv_req',
    rhs=space_required[5]))

#the total of Jul through Aug leases and leases made in Feb/Mar/Apr/May/Jul that include apr must be greater than or equal to 
#the space required for Jul
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(allocation[6,:-4])+pulp.lpSum(allocation[1,5])+pulp.lpSum(allocation[2,4:])+
        pulp.lpSum(allocation[3,3:-1])+pulp.lpSum(allocation[4,2:-2])+pulp.lpSum(allocation[5,1:-3]),
    sense=pulp.LpConstraintGE,
    name='Jul_inv_req',
    rhs=space_required[6]))

#the total of Aug leases and leases made in Mar/Apr/May/Jul that include apr must be greater than or equal to 
#the space required for Aug
model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(allocation[7,0])+pulp.lpSum(allocation[2,5])+pulp.lpSum(allocation[3,4])+pulp.lpSum(allocation[4,3])+
                pulp.lpSum(allocation[5,2])+pulp.lpSum(allocation[6,1]),
    sense=pulp.LpConstraintGE,
    name='Aug_inv_req',
    rhs=space_required[7]))

objective = pulp.lpSum([allocation[:,i]*cost for i,cost in enumerate(costs)])

model.setObjective(objective)

model.solve()

#if the model is optimal, print the objective value and the production matrix
if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    output = np.array([allocation[i,j].value() for i in range(n_months) for j in range(n_lease)]).reshape(n_months, n_lease)

    print(pd.DataFrame(output, columns=lease, index=months))
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')



status: 1, Optimal
objective: $81,300
       1     2    3    4    5     6
Jan  0.0  18.0  3.0  1.0  2.0   0.0
Feb  0.0   0.0  0.0  0.0  0.0   4.0
Mar  0.0   0.0  0.0  0.0  0.0  21.0
Apr  0.0   0.0  0.0  0.0  0.0   0.0
May  0.0   0.0  0.0  0.0  0.0   0.0
Jun  0.0   0.0  0.0  0.0  0.0   0.0
Jul  2.0   7.0  0.0  0.0  0.0   0.0
Aug  0.0   0.0  0.0  0.0  0.0   0.0


In [45]:
model

lease_optimization:
MINIMIZE
400*L_Apr1 + 780*L_Apr2 + 1140*L_Apr3 + 1480*L_Apr4 + 1800*L_Apr5 + 2100*L_Apr6 + 400*L_Aug1 + 780*L_Aug2 + 1140*L_Aug3 + 1480*L_Aug4 + 1800*L_Aug5 + 2100*L_Aug6 + 400*L_Feb1 + 780*L_Feb2 + 1140*L_Feb3 + 1480*L_Feb4 + 1800*L_Feb5 + 2100*L_Feb6 + 400*L_Jan1 + 780*L_Jan2 + 1140*L_Jan3 + 1480*L_Jan4 + 1800*L_Jan5 + 2100*L_Jan6 + 400*L_Jul1 + 780*L_Jul2 + 1140*L_Jul3 + 1480*L_Jul4 + 1800*L_Jul5 + 2100*L_Jul6 + 400*L_Jun1 + 780*L_Jun2 + 1140*L_Jun3 + 1480*L_Jun4 + 1800*L_Jun5 + 2100*L_Jun6 + 400*L_Mar1 + 780*L_Mar2 + 1140*L_Mar3 + 1480*L_Mar4 + 1800*L_Mar5 + 2100*L_Mar6 + 400*L_May1 + 780*L_May2 + 1140*L_May3 + 1480*L_May4 + 1800*L_May5 + 2100*L_May6 + 0
SUBJECT TO
Jan_inv_req: L_Jan1 + L_Jan2 + L_Jan3 + L_Jan4 + L_Jan5 + L_Jan6 >= 24

Feb_inv_req: L_Feb1 + L_Feb2 + L_Feb3 + L_Feb4 + L_Feb5 + L_Feb6 + L_Jan2
 + L_Jan3 + L_Jan4 + L_Jan5 + L_Jan6 >= 28

Mar_inv_req: L_Feb2 + L_Feb3 + L_Feb4 + L_Feb5 + L_Feb6 + L_Jan3 + L_Jan4
 + L_Jan5 + L_Jan6 + L_Mar1 + L_Mar2 +